# 1. Introduction to the Data Set
Over the past two missions, we learned about how decision trees are constructed. We used a modified version of ID3, which is a bit simpler than the most common tree building algorithms, C4.5 and CART. The basics are the same, however, so we can apply what we learned about how decision trees work to any tree construction algorithm.

In this mission, we'll learn about when to use decision trees, and how to use them most effectively.

We'll continue using the 1994 census data on U.S. incomes we worked with in the previous mission. It contains information on marital status, age, type of work, and more. The target column, high_income, indicates an income of less than or equal to 50k a year (0), or more than 50k a year (1).

You can download the data from the University of California, Irvine's website. http://archive.ics.uci.edu/ml/datasets/Adult

# 2. Using Decision Trees with scikit-learn

We can use the scikit-learn package to fit a decision tree. The interface is very similar to other algorithms we've fit in the past.

We use the DecisionTreeClassifier class for classification problems, and DecisionTreeRegressor for regression problems. The sklearn.tree package includes both of these classes.

In this case, we're predicting a binary outcome, so we'll use a classifier.

The first step is to train the classifier on the data. We'll use the fit method on a classifier to do this.

**Instructions:**
Fit clf to the income data.
- Pass in income[columns] so that we only use the named columns as predictors.
- The target is the high_income column.

In [10]:
import pandas as pd
income = pd.read_csv('income2.csv')
income.shape

income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
high_income       32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [12]:
for name in ["workclass","education", "marital_status", "occupation", "relationship", "race", "sex", "native_country", "high_income"]:
    col = pd.Categorical.from_array(income[name])
    income[name] = col.codes

C:\Users\mciniello\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: Categorical.from_array is deprecated, use Categorical instead
  from ipykernel import kernelapp as app


In [14]:
from sklearn.tree import DecisionTreeClassifier

# A list of columns to train with
# We've already converted all columns to numeric
columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

# Instantiate the classifier
# Set random_state to 1 to make sure the results are consistent
clf = DecisionTreeClassifier(random_state=1)

# We've already loaded the variable "income," which contains all of the income data

clf.fit(income[columns], income["high_income"])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=1, splitter='best')

# 3. Splitting the Data into Train and Test Sets

Now that we've fit a model, we can make predictions. We'll want to split our data into training and testing sets first. If we don't, we'll be making predictions on the same data that we train our algorithm with. This leads to overfitting, and will make our error appear lower than it is.

While we covered overfitting in more depth in previous missions, here's a quick recap. If you memorize how to perform three specific addition problems (2+2, 3+6, 3+3), you'll get those specific problems correct every time.

On the other hand, if someone asks you what 4+4 is, you won't know how to do it, because you don't know the rules of addition. If you learn the rules of addition, you'll get problems wrong sometimes (because operations like 3443343434+24344343 can be hard to do mentally). Even so, you'll be able to do any problem, and you'll get most of them right.

The first example represents overfitting, where you memorize the details of the training set, but can't generalize to new examples you're asked to make predictions on.

We can avoid overfitting by always making predictions and evaluating error on data that we haven't trained our algorithm with. This will show us when we're overfitting by giving us a realistic error on data that the algorithm hasn't seen before.

We can split the data by shuffling the order of the dataframe, then selecting certain rows to include in the training set, and certain rows to include in the testing set.

In this case, we'll make 80% of our rows training data, and the rest testing data.

**Instructions:**

All of the rows in income with a position up to train_max_row (but not including it) will be part of the training set.
- Make a new dataframe called train containing all of these rows.
- Make a dataframe called test containing all of the rows with a position greater than or equal to train_max_row.

In [16]:
import numpy
import math

# Set a random seed so the shuffle is the same every time
numpy.random.seed(1)

# Shuffle the rows  
# This permutes the index randomly using numpy.random.permutation
# Then, it reindexes the dataframe with the result
# The net effect is to put the rows into random order
income = income.reindex(numpy.random.permutation(income.index))

train_max_row = math.floor(income.shape[0] * .8)
train = income.iloc[:train_max_row]
test = income.iloc[train_max_row:]

# 4. Evaluating Error with AUC

While there are many methods for evaluating error with classification, we'll use AUC, which we've already covered extensively in the machine learning material. AUC ranges from 0 to 1, so it's ideal for binary classification. The higher the AUC, the more accurate our predictions.

We can compute AUC with the roc_auc_score function from sklearn.metrics. This function takes in two parameters:

- y_true: true labels
- y_score: predicted labels
It then calculates and returns the AUC value.

**Instructions:**

- Compute the AUC between predictions and the high_income column of test, and assign the result to error.
- Use the print function to display error.

In [18]:
from sklearn.metrics import roc_auc_score

clf = DecisionTreeClassifier(random_state=1)
clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
error = roc_auc_score(test["high_income"], predictions)
print(error)

0.69237495612


# 5. Computing Error on the Training Set

The AUC for the predictions on the testing set is about .694. Let's compare this against the AUC for predictions on the training set to see if the model is overfitting.

It's normal for the model to predict the training set better than the testing set. After all, it has full knowledge of that data and the outcomes. However, if the AUC between training set predictions and actual values is significantly higher than the AUC between test set predictions and actual values, it's a sign that the model may be overfitting.

In [20]:
predictions = clf.predict(train[columns])
print(roc_auc_score(train["high_income"], predictions))

0.945777105887


# 6. Decision Tree Overfitting

Our AUC on the training set was .947, and the AUC on the test set was .694. There's no hard and fast rule on when overfitting is occurring, but our model is predicting the training set much better than the test set. Splitting the data into training and testing sets doesn't prevent overfitting -- it just helps us detect and fix it.

Based on our AUC measurements, it appears that we are in fact overfitting. Let's take a closer look at why decision trees might overfit.

In the last mission, we looked at this data:

    high_income    age    marital_status
    0              20     0
    0              60     2
    0              40     1
    1              25     1
    1              35     2
    1              55     1

Here's the full diagram for the decision tree we can build from it:

![](Picture1.png)

This tree predicts all of our values perfectly. It will always get a right answer on the training set, but this is equivalent to memorizing the rules of addition. While we've built our tree in such a way that it can perfectly predict the training set, the way it's constructed doesn't make sense when we take a step back.

That's because the tree above is saying that:

    If you're under 22.5 years old, you have a low income
    If you're 22.5 - 37.5, you have a high income
    If you're 37.5 - 47.5, you have a low income
    If you're 47.5 to 55, you have a high income
    Finally, if you're above 55, you have a low income
    
These rules are very specific to the training set.

Think about the problem with a real-world lens. Does it make sense to predict that someone who's 20 has a low income, someone who's 25 has a high income, and someone who's 40 has a low income? Intuitively, we know that younger people tend to earn less, middle-aged people earn more, and people who have retired earn less.

Our tree has created so many age-based splits in an attempt to perfectly predict everyone's income that each split is effectively meaningless.

Here's a tree that matches up with our intuition better:

![](Picture2.png)

All we've done is "pruned" the tree, and removed some of the lower leaves. We've turned some of the higher-level nodes into leaves instead.

The tree above makes more intuitive sense. If you're under 25, we predict low income. If you're between 25 and 55, we predict high income (the .66 rounds up to 1). If you're above 55, we predict low income.

This version actually has lower accuracy on our training set, but will generalize to new examples better because it matches reality more closely.

Trees overfit when they have too much depth and make overly complex rules that match the training data, but aren't able to generalize well to new data. This may seem to be a strange principle at first, but the deeper a tree is, the worse it typically performs on new data.

# 7. Reducing Overfitting with a Shallower Tree

There are three main ways to combat overfitting:

1. "Prune" the tree after we build it to remove unnecessary leaves.
2. Use ensembling to blend the predictions of many trees.
3. Restrict the depth of the tree while we're building it.

While we'll explore all of these, we'll look at the third method first.

Limiting tree depth during the building process will result in more general rules. This prevents the tree from overfitting.

We can restrict tree depth by adding a few parameters when we initialize the DecisionTreeClassifier class:

- max_depth - Globally restricts how deep the tree can go
- min_samples_split - The minimum number of rows a node should have before it can be split; if this is set to 2, for example, then nodes with 2 rows won't be split, and will become leaves instead
- min_samples_leaf - The minimum number of rows a leaf must have
- min_weight_fraction_leaf - The fraction of input rows a leaf must have
- max_leaf_nodes - The maximum number of total leaves; this will cap the count of leaf nodes as the tree is being built

Some of these parameters aren't compatible, however. For example, we can't use max_depth and max_leaf_nodes together.

Now that we know what to tweak, let's improve our model.

**Instructions**

- Set min_samples_split to 13 when creating the DecisionTreeClassifier.
- Make predictions on the training set, compute the AUC, and assign it to train_auc.
- Make predictions on the test set, compute the AUC, and assign it to test_auc.

In [22]:
# Decision trees model from the last screen
clf = DecisionTreeClassifier(random_state=1)
clf = DecisionTreeClassifier(min_samples_split=13, random_state=1)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
print(train_auc)

0.710080195443
0.836411006149


# 8. Tweaking Parameters to Adjust AUC

By setting min_samples_split to 13, we managed to boost the test AUC from .694 to .700. The training set AUC decreased from .947 to .843, showing that the model we built was less overfit to the training set than before:

    settings	train AUC	test AUC
    default	0.947	0.694
    min_samples_split: 13	0.843	0.700

Let's play around with parameters some more.

**Instructions**

- Set max_depth to 7 and min_samples_split to 13 when creating the DecisionTreeClassifier.
- Make predictions on the training set, compute the AUC, and assign it to train_auc.
- Make predictions on the test set, compute the AUC, and assign it to test_auc.

In [24]:
clf = DecisionTreeClassifier(random_state=1, min_samples_split=13, max_depth=7)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
print(train_auc)

0.75043213051
0.749285076088


# 9. Tweaking Tree Depth to Adjust AUC

We just improved the AUC again! The test set AUC increased to .744, while the training set AUC decreased to .748:

    settings	train AUC	test AUC
    default (min_samples_split: 2, max_depth: None):	0.947	0.694
    min_samples_split: 13:	0.843	0.700
    min_samples_split: 13, max_depth: 7:	0.748	0.7744
    
We aren't overfitting anymore because both AUC values are about the same. Let's tweak the parameters more aggressively and see what happens!

**Instructions**

- Set max_depth to 2 and min_samples_split to 100 when creating the DecisionTreeClassifier.
- Make predictions on the training set, compute the AUC, and assign it to train_auc.
- Make predictions on the test set, compute the AUC, and assign it to test_auc.

In [26]:
clf = DecisionTreeClassifier(random_state=1, min_samples_split=100, max_depth=2)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
print(train_auc)

0.666779902304
0.659499251491


# 10. Underfitting in Simplistic Trees
Our accuracy went down on the last screen, relative to the screen before it:

settings	train AUC	test AUC
default (min_samples_split: 2, max_depth: None):	0.947	0.694
min_samples_split: 13:	0.843	0.700
min_samples_split: 13, max_depth: 7:	0.748	0.7744
min_samples_split: 100, max_depth: 2:	0.662	0.655

This is because we're now underfitting. Underfitting is what occurs when our model is too simple to explain the relationships between the variables.

# 11. The Bias-Variance Tradeoff
This is known as the bias-variance tradeoff. Imagine that we take a random sample of training data and create many models. If the models' predictions for the same row are far apart from each other, we have high variance. Imagine this time that we take a random sample of the training data and create many models. If the models' predictions for the same row are close together but far from the actual value, then we have high bias.

High bias can cause underfitting -- if a model is consistently failing to predict the correct value, it may be that it's too simple to model the data faithfully.

High variance can cause overfitting. If a model varies its predictions significantly based on small changes in the input data, then it's likely fitting itself to quirks in the training data, rather than making a generalizable model.

We call this the bias-variance tradeoff because decreasing one characteristic will usually increase the other. This is a limitation of all machine learning algorithms. If you'd like to read more about the tradeoff, check out programmer Scott Fortmann-Roe's post on this topic.

Decision trees typically suffer from high variance. The entire structure of a decision tree can change if we make a minor alteration to its training data. By restricting the depth of the tree, we increase the bias and decrease the variance. If we restrict the depth too much, we increase bias to the point where it will underfit.

You'll generally need to use your intuition and manually tweak parameters to get the "right" fit.

# 12. Exploring Decision Tree Variance

We can induce variance and see what happens with a decision tree. To introduce noise into the data, we'll add a column of random values. A model with high variance (like a decision tree) will pick up on this noise, and overfit to it. This is because models with high variance are very sensitive to small changes in input data.

**Instructions**

- Fit the classifier to the training data.
- Make predictions on the training set, compute the AUC, and assign it to train_auc.
- Make predictions on the test set, compute the AUC, and assign it to test_auc.

In [28]:
numpy.random.seed(1)

# Generate a column containing random numbers from 0 to 4
income["noise"] = numpy.random.randint(4, size=income.shape[0])

# Adjust "columns" to include the noise column
columns = ["noise", "age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

# Make new train and test sets
train_max_row = math.floor(income.shape[0] * .8)
train = income.iloc[:train_max_row]
test = income.iloc[train_max_row:]

# Initialize the classifier
clf = DecisionTreeClassifier(random_state=1)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
print(train_auc)

0.698333590245
0.975645926216


# 13. Pruning Leaves to Prevent Overfitting

As you can see above, the random noise column causes significant overfitting. Our test set accuracy decreases to .691, and our training set accuracy increases to .975.

One way to prevent overfitting is to block the tree from growing beyond a certain depth (we tried this before). Another technique is called pruning. Pruning involves building a full tree, and then removing the leaves that don't add to prediction accuracy. Pruning prevents a model from becoming overly complex. It can result in a simpler model that has higher accuracy on the testing set.

Data scientists use pruning less often than parameter optimization (what we just did) and ensembling. It's still an important technique, though, and we'll cover it in more depth down the line.

# 14. Knowing When to Use Decision Trees

Let's go over the main advantages and disadvantages of using decision trees. The main advantages of using decision trees is that they're:

- Easy to interpret
- Relatively fast to fit and make predictions
- Able to handle multiple types of data
- Able to pick up nonlinearities in data, and usually fairly accurate

The main disadvantage of using decision trees is their tendency to overfit.

Decision trees are a good choice for tasks where it's important to be able to interpret and convey why the algorithm is doing what it's doing.

The most powerful way to reduce decision tree overfitting is to create ensembles of trees. The random forest algorithm is a popular choice for doing this. In cases where prediction accuracy is the most important consideration, random forests usually perform better.

In the next mission, we'll explore the random forest algorithm in greater depth.